# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [45]:
# YOUR CHANGES HERE
#imports
#!pip install statsmodels 
import pandas as pd
import numpy as np
import math
from pathlib import Path
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [6]:
PRICES = Path("strawberry-prices.tsv")
BACKTEST = Path("strawberry-backtest.tsv")

In [7]:
df = pd.read_csv(PRICES, sep="\t")
df.columns = [c.lower().strip() for c in df.columns]
date_col = next(c for c in df.columns if c in ("date","month","period","month_start"))
price_col = next(c for c in df.columns if c in ("price","value","avg_price","strawberry_price","price_usd"))

In [8]:
df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col).set_index(date_col)

In [9]:
idx = pd.date_range(df.index.min(), df.index.max(), freq="MS")
y = df.reindex(idx)[price_col].astype(float).interpolate("linear")
y.index.name = "month"

In [10]:
y_train = y[y.index.year <= 2023]
y_2024  = y[y.index.year == 2024]

In [ ]:
# exponential smoothing model 
model = ExponentialSmoothing(
    y_train,
    trend="add",
    seasonal="mul",
    seasonal_periods=12,
    initialization_method="estimated",
).fit(optimized=True)

In [ ]:
# forecast 
steps = 12
forecast_2024 = model.forecast(steps)
forecast_2024.index = pd.date_range(pd.Timestamp("2024-01-01"), periods=steps, freq="MS")

In [15]:
backtest = pd.DataFrame({
    "month": forecast_2024.index.strftime("%Y-%m"),
    "price": np.round(forecast_2024.values, 4)
})
backtest.to_csv(BACKTEST, sep="\t", index=False)

In [33]:
backtest

,month,price
0,2024-01,4.5401
1,2024-02,4.2593
2,2024-03,3.8352
3,2024-04,3.9481
4,2024-05,3.6744
5,2024-06,3.4130
6,2024-07,3.3291
7,2024-08,3.6370
8,2024-09,3.7362
9,2024-10,4.0860


Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [16]:
# YOUR CHANGES HERE

BACKTEST_ACCURACY = Path("backtest-accuracy.tsv")

In [17]:
prices = pd.read_csv(PRICES, sep="\t")
prices.columns = [c.lower().strip() for c in prices.columns]
date_col  = next(c for c in prices.columns if c in ("date","month","period","month_start"))
price_col = next(c for c in prices.columns if c in ("price","value","avg_price","strawberry_price","price_usd"))

In [18]:
prices[date_col] = pd.to_datetime(prices[date_col])
prices = prices.sort_values(date_col).set_index(date_col)

In [ ]:
# 2024 data
idx_2024 = pd.date_range("2024-01-01", periods=12, freq="MS")
actual_2024 = prices.loc[prices.index.year == 2024, price_col].reindex(idx_2024)

In [ ]:
# results from first prediction
backtest_pred = pd.read_csv(BACKTEST, sep="\t")
backtest_pred.columns = [c.lower().strip() for c in backtest_pred.columns]
backtest_pred["month"] = pd.to_datetime(backtest_pred["month"] + "-01")
backtest_pred = backtest_pred.set_index("month").sort_index()

In [ ]:
# calculate residuals

resid_df = pd.DataFrame({
    "actual": actual_2024,
    "pred": backtest_pred["price"]
}).dropna()

residuals = resid_df["actual"] - resid_df["pred"]

In [ ]:
# mean and std of residuals
mean_resid = residuals.mean()
std_resid = residuals.std(ddof=1)

In [23]:
backtest_acc = pd.DataFrame({"mean": [mean_resid], "std": [std_resid]})
backtest_acc.to_csv(BACKTEST_ACCURACY, sep="\t", index=False)

In [32]:
backtest_acc

,mean,std
0,-0.165517,0.265628


Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [24]:
# YOUR CHANGES HERE

STRAWBERRY_FORECAST = Path("strawberry-forecast.tsv")

In [25]:
df = pd.read_csv(PRICES, sep="\t")
df.columns = [c.lower().strip() for c in df.columns]

In [26]:
date_col  = next(c for c in df.columns if c in ("date","month","period","month_start"))
price_col = next(c for c in df.columns if c in ("price","value","avg_price","strawberry_price","price_usd"))

In [27]:
df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col).set_index(date_col)

In [28]:
idx = pd.date_range(df.index.min(), df.index.max(), freq="MS")
y = df.reindex(idx)[price_col].astype(float).interpolate("linear")
y.index.name = "month"

In [ ]:
# forecast 2025
y_2024 = y[y.index.year == 2024]

forecast_index = pd.date_range("2025-01-01", periods=12, freq="MS")
forecast_2025 = pd.Series(y_2024.values, index=forecast_index)

In [30]:
stb_forecast = pd.DataFrame({
    "month": forecast_2025.index.strftime("%Y-%m"),
    "price": np.round(forecast_2025.values, 4)
})
stb_forecast.to_csv(STRAWBERRY_FORECAST, sep="\t", index=False)

In [31]:
stb_forecast

,month,price
0,2025-01,5.055
1,2025-02,4.264
2,2025-03,3.742
3,2025-04,3.576
4,2025-05,3.237
5,2025-06,2.977
6,2025-07,3.116
7,2025-08,3.347
8,2025-09,3.742
9,2025-10,3.718


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [34]:
# YOUR CHANGES HERE

TIMINGS = Path("timings.tsv")

In [44]:
capital = 1000000.0
freeze_cost = 0.20
storage_cost_per_month = 0.10
discount = 0.10

In [43]:
straw_forecast = pd.read_csv(STRAWBERRY_FORECAST, sep="\t")
straw_forecast.columns = [c.lower().strip() for c in straw_forecast.columns]
straw_forecast["month"] = pd.to_datetime(straw_forecast["month"] + "-01")
straw_forecast = straw_forecast.set_index("month").sort_index()

In [ ]:
rows = []
months = straw_forecast.index.tolist()

for i, buy_dt in enumerate(months):
    buy_price = float(straw_forecast.loc[buy_dt, "price"])
    for j, sell_dt in enumerate(months):
        if j < i:
            continue
        # full months between buying and selling
        months_diff = (sell_dt.year - buy_dt.year) * 12 + (sell_dt.month - buy_dt.month)
        # cost per pint
        total_cost_per_pint = buy_price + freeze_cost + storage_cost_per_month * months_diff

        # affordable pints
        pints = math.floor(capital / total_cost_per_pint)

        # forecast sell price
        sell_price = float(straw_forecast.loc[sell_dt, "price"])
        # 10% reduction
        revenue_per_pint = sell_price * (1 - discount)
        # profit per pint
        profit_per_pint  = revenue_per_pint - total_cost_per_pint
        # expected profit
        expected_profit  = profit_per_pint * pints

        rows.append({
            "buy_month": buy_dt.strftime("%Y-%m"),
            "sell_month": sell_dt.strftime("%Y-%m"),
            "pints_purchased": int(pints),
            "expected_profit": round(expected_profit, 2)
        })

In [47]:
timings = pd.DataFrame(rows).sort_values("expected_profit", ascending=False).reset_index(drop=True)
timings.to_csv(TIMINGS, sep="\t", index=False)

In [48]:
timings

,buy_month,sell_month,pints_purchased,expected_profit
0,2025-06,2025-12,264760,157346.87
1,2025-07,2025-12,262054,145518.59
2,2025-08,2025-12,253356,107498.95
3,2025-06,2025-11,271960,81859.96
4,2025-07,2025-11,269106,70505.77
...,...,...,...,...
73,2025-01,2025-10,162469,-456342.93
74,2025-01,2025-05,176834,-484825.78
75,2025-01,2025-08,167926,-494155.84
76,2025-01,2025-07,170794,-521024.18


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [60]:
# YOUR CHANGES HERE
CHECK = Path("check.tsv")

timings = pd.read_csv(TIMINGS, sep="\t")
strawb_forecast = pd.read_csv(STRAWBERRY_FORECAST, sep="\t")
backtest_accuracy = pd.read_csv(BACKTEST_ACCURACY, sep="\t")

In [53]:
strawb_forecast["month"] = pd.to_datetime(strawb_forecast["month"] + "-01")
strawb_forecast = strawb_forecast.set_index("month").sort_index()
std_resid = float(backtest_accuracy.loc[0, "std"])

In [54]:
freeze_cost = 0.20
storage_cost_per_month = 0.10
discount = 0.10

In [ ]:
best = timings.sort_values("expected_profit", ascending=False).iloc[0]
buy_month  = str(best["buy_month"])
sell_month = str(best["sell_month"])
pints      = int(best["pints_purchased"])

buy_dif  = pd.to_datetime(buy_month + "-01")
sell_dif = pd.to_datetime(sell_month + "-01")

buy_price  = float(strawb_forecast.loc[buy_dif,  "price"])
sell_price = float(strawb_forecast.loc[sell_dif, "price"])

In [ ]:
months_diff = (sell_dif.year - buy_dif.year) * 12 + (sell_dif.month - buy_dif.month)

In [ ]:
# cost per pint & revenue
total_cost_per_pint = buy_price + freeze_cost + storage_cost_per_month * months_diff
revenue_pp = sell_price * (1 - discount)

In [ ]:
# profit calculation
profit_pp = revenue_pp - total_cost_per_pint
profit    = round(profit_pp * pints, 2)

In [ ]:
# profit considering residual 
after_sell_price  = max(sell_price - std_resid, 0.0)
after_revenue_pp  = after_sell_price * (1 - discount)
after_profit_pp   = after_revenue_pp - total_cost_per_pint
one_std_profit       = round(after_profit_pp * pints, 2)

In [ ]:
check = pd.DataFrame({
    "best_profit":   [profit],
    "one_std_profit":[one_std_profit]
})

check.to_csv(CHECK, sep="\t", index=False)

In [62]:
check

,best_profit,one_std_profit
0,157346.87,94051.9


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.